# Feature Engineering

For some definitions, handling outliers and missing values, scaling, and encoding may be considered feature engineering. Here we'll draw a distinction between data preparation, data preprocessing, and feature engineering.

- **data preparation**: the basic data cleaning necessary to get our data ready for exploration/analysis, e.g. correcting data types, fixing typos
- **data preprocessing**: further data transformation done for the sake of modeling, as oppsoed to exploration/analysis, e.g. scaling, imputing, encoding
- **feature engineering**: adding, combining, or removing features; usually with the help of domain knowledge

Feature engineering can happen as part of data exploration or modeling, and engineered featured are also commonly explored.

Some examples of feature engineering by this definition:

- domain-based conversion (example: farenheit to celsius, BMI calculation, log transformation)
- domain based cutoffs (example: age >= 18 = is_adult; also dates)

- add / subtract (example: zillow dataset: beds + baths = room_count; total_sqft - 200 * bedrooms - 40 * bathrooms = living_area)
- combine as booleans as a count (example: telco_churn: streaming + backups + ...  = service_count)
- multiply / divide (example: tips dataset: total_bill / size = price_per_person)
- ratios (example: tips dataset: tip / total_bill = tip percentage)

Simplify!

- categorical with many unique values to top 3 + "Other"
- categorical to boolean: pool count -> has pool
- continous -> categorical via binning (aka quantization or discretization) (example: income -> high, medium, low earner)

In this lesson we'll cover some *automated* **feature selection** methods, that is, methods for determining which features are the most important.

- SelectKBest
- Recursive Feature Elimination
- Sequential Feature Selection

## Setup

In [3]:
import numpy as np
import pandas as pd
import wrangle
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector

In [7]:
df = pd.read_csv('student_grades - Sheet1.csv')

In [11]:
df.isna().sum()

student_id     0
exam1          1
exam2          0
exam3          1
final_grade    0
dtype: int64

In [22]:
df.exam1 = df.exam1.fillna(78.4)
df.exam3 = df.exam3.fillna(84.24)
df.isna().sum()

student_id     0
exam1          0
exam2          0
exam3          0
final_grade    0
dtype: int64

In [23]:
train, validate, test = wrangle.split(df)

train.shape:(58, 5)
validate.shape:(25, 5)
test.shape:(21, 5)


In [24]:
train.shape, validate.shape, test.shape

((58, 5), (25, 5), (21, 5))

## Select K Best

- looks at each feature in isolation against the target based on correlation
- fastest of all approaches covered in this lesson
- doesn't consider feature interactions
- After fitting: `.scores_`, `.pvalues_`, `.get_support()`, and `.transform`

In [35]:
x_train = train.drop(columns='final_grade')

y_train = train.final_grade

In [36]:
# make the thing
kbest = SelectKBest(f_regression, k=2)
# fit the thing
_ = kbest.fit(x_train, y_train)

In [38]:
# statistical f-value:
kbest.scores_
#p value: 
kbest.pvalues_

array([5.54053992e-01, 2.98729511e-42, 2.91888370e-25, 2.18881531e-32])

In [39]:
# get-support() will output a boolean mask to tell me which features were selected
# we can apply this mask to the columns in our original dataframe
x_train.columns[kbest.get_support()]

Index(['exam1', 'exam3'], dtype='object')

In [40]:
# kbest transform will convert our information to the selected
# feature subspace
# ****buuuuuut, its just a numpy array
x_train.iloc[:, kbest.get_support()].head()

,exam1,exam3
90,70.0,78.0
15,85.0,87.0
55,83.0,86.0
75,58.0,70.0
25,70.0,78.0


## RFE

- Recursive Feature Elimination
- Progressively eliminate features based on importance to the model
- Requires a model with either a `.coef_` or `.feature_importances_` property
- After fitting: `.ranking_`, `.get_support()`, and `.transform()`

In [41]:
from sklearn.linear_model import LinearRegression

In [42]:
# establish a model for RFE to use (make a model thing):
# linear model works the same as in anny other context here, so do all the preprocessing
# that you would normally do to those features before you get to this point!
model = LinearRegression()

In [43]:
# make an RFE thing:
rfe = RFE(model, n_features_to_select=2)

In [44]:
# fit the RFE:
rfe.fit(x_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [46]:
rfe.ranking_

array([3, 1, 2, 1])

In [47]:
x_train.columns

Index(['student_id', 'exam1', 'exam2', 'exam3'], dtype='object')

In [48]:
pd.DataFrame({
    'rfe_ranking': rfe.ranking_}, index=x_train.columns)

,rfe_ranking
student_id,3
exam1,1
exam2,2
exam3,1


In [53]:
rfe.get_support()

array([False,  True, False,  True])

In [51]:
rfe.support_

array([False,  True, False,  True])

In [56]:
pd.DataFrame(rfe.transform(x_train),
            columns=x_train.columns[rfe.get_support()],
            index=x_train.index).head()

,exam1,exam3
90,70.0,78.0
15,85.0,87.0
55,83.0,86.0
75,58.0,70.0
25,70.0,78.0


## Sequential Feature Selector

- progressively adds features based on cross validated model performance
- forwards: start with 0, add the best additional feature until you have the desired number
- backwards: start with all features, remove the worst performing until you have the desired number
- After fitting: `.support_`, `.transform`

## Conclusion

- Simpler models handle change + variability better
- Use RFE to narrow down your features and find the best ones, if your dataset is large (> 1GB; `df.info()`) use select k best instead
- Remember: feature engineering is much more than feature selection!